##  hopfield network to detect english alphabet images

In [ ]:
import numpy as np
from PIL import Image, ImageFont
import random
import matplotlib.image as mpimg
import os

In [ ]:
def train(patterns):
    weights = np.zeros((patterns.shape[1], patterns.shape[1]))
    for p in patterns:
        for i in range(weights.shape[0]):
            for j in range(weights.shape[1]):
                if i == j:
                    weights[i, j] = 0
                else:
                    weights[i, j] += p[i] * p[j] / len(p)
                    #weights[j, i] = weights[i, j]
    return weights


def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0


def test(input_pattern, weights):
    before_iterate = np.copy(input_pattern)
    after_iterate = np.copy(input_pattern)
    n = 0
    while True:
        n += 1
        if n % 10 == 0:
            print(n)
        before_iterate = np.copy(after_iterate)
        # print(after_iterate.shape[0])
        for idx in range(after_iterate.shape[0]):
            result = np.dot(after_iterate, weights[idx])
            #if after_iterate[idx] != sign(result):
                #print(idx, n)
                #print(after_iterate[idx], sign(result))
            after_iterate[idx] = sign(result)
        if np.array_equal(before_iterate, after_iterate) is True:
            break
        if n >= 100:
            print("this is not going to converge.")
            break
    print(f"number of iteration: {n}")
    print("after_iterate", np.array_equal(after_iterate, input_pattern))
    print(energy_function(weights, after_iterate))
    return after_iterate


def which_number(arr, patterns):
    # print(patterns[8][1])
    for i in range(len(patterns)):
        # print(len(patterns[i]))
        if np.array_equal(patterns[i], arr):
        #if patterns[0][i] != arr[i]:
         #   print(i)
          #  print(arr[i])
           # print(patterns[2][i])
            return i


In [ ]:
def energy_function(weights, vector):
    energy = 0
    energy = -sum([weights[i, j] * vector[i] * vector[j] for i in range(len(vector)) for j in range(len(vector))])
    return energy

In [ ]:
def save_images():
    sizes = [16, 32, 64]
    for font_size in sizes:
        # create folder if not exists
        if not os.path.exists(f"image_patterns/font_size_{font_size}"):
            os.mkdir(f"image_patterns/font_size_{font_size}")
          
        # load first 10 characters
        font = ImageFont.truetype("arial.ttf", font_size)
        for char in "ABCDEFGHIJ":
            im = Image.Image()._new(font.getmask(char))
            im.save(f"image_patterns\\font_size_{font_size}\\{char}_{font_size}.bmp")

            
def generate_noisy_images():
    if not os.path.exists(f"image_noisy"):
                    os.mkdir(f"image_noisy")
    sizes = [16, 32, 64]
    noise_percents = [0.1, 0.3, 0.6]
    for font_size in sizes:
        for char in "ABCDEFGHIJ":
            img = mpimg.imread(f"image_patterns\\font_size_{font_size}\\{char}_{font_size}.bmp")
            width = len(img)
            height = len(img[0])
            img_list = [-1 if img[i, j] < 100 else 1 for i in range(len(img)) for j in range(len(img[0]))]
            for percent in noise_percents:
                if not os.path.exists(f"image_noisy\\font_size_{font_size}_p{percent}"):
                    os.mkdir(f"image_noisy\\font_size_{font_size}_p{percent}")
            
                for i in range(int(percent * 200)):
                    rnd = random.randrange(1, width * height)
                    img_list[rnd] = not img_list[rnd]
                noisy_list = [255 if img_list[i] == 1 else 0 for i in range(len(img_list))]
                noisy_array = np.array(noisy_list).reshape(width, height)
                mpimg.imsave(f"image_noisy\\font_size_{font_size}_p{percent}\\{char}_{font_size}_p{percent}.bmp", noisy_array)
                #im.save(f"image_noisy\\font_size_{font_size}_p{percent}\\{char}_{font_size}_p{percent}.bmp")
    return

                
save_images()
generate_noisy_images()






# load patterns
patterns = []
for c in "ABCD":
    #im = Image.open(f"{c}_16.bmp").convert('RGB')
    #px = im.load()
    img = mpimg.imread(f"{c}_16.bmp")
    width, height = im.size

    p = []
    p = [-1 if img[i, j] < 200 else 1 for i in range(len(img)) for j in range(len(img[0]))]

    for i in range(200 - len(p)):
        p.append(1)
    patterns.append(p)
    
    # print(px)

    
# ******** load input pattern ************
weights = train(np.array(patterns))

for c in "ABCDEFGHIJ":
    
    img = mpimg.imread(f"{c}_16.bmp")

    input_pattern = [-1 if img[i, j] < 200 else 1 for i in range(len(img)) for j in range(len(img[0]))]
    for i in range(200 - len(input_pattern)):
        input_pattern.append(1)
    print(c, energy_function(weights, input_pattern))
    random_pick = np.random.rand((10))
    for _ in range(20):
        r = random.randrange(0,200)
    #print(r)
        input_pattern[r] = not input_pattern[r]
    print(energy_function(weights, random_pick))
    
    output = test(np.array(input_pattern), weights)
    print("output, intput", np.array_equal(output, np.array(input_pattern)))
#print(type(output), type(np.array(input_pattern)))
    print(which_number(output, patterns))